In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from code.modelhelper import *

c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dty

In [2]:
BATCH_SIZE = 512
N_CLASS = 2
EPOCHS = 200

In [3]:
x = np.load("data/xy/x_feat_630.npz")["arr_0"]
y = np.load("data/xy/y_onehot_45_15.npy")
x.shape, y.shape

((60000, 630), (60000, 2))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=11)

# MLP w/ Engineered Features

In [5]:
model_input = Input(shape=(x.shape[1], ), name="m1_input")
H = Dense(50, activation='relu', name="m1_dense_1")(model_input)
H = Dense(100, activation="relu", name="m1_dense_2")(H)
H = Dense(150, activation="relu", name="m1_dense_3")(H)
H = Dense(200, activation="relu", name="m1_dense_4")(H)
H = Dropout(0.4, name="m1_dropout_1")(H)
model_output = Dense(N_CLASS, activation="softmax", name="m1_output")(H)

model = Model(inputs=model_input, outputs=model_output)
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy']
)

es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
m1_input (InputLayer)        [(None, 630)]             0         
_________________________________________________________________
m1_dense_1 (Dense)           (None, 50)                31550     
_________________________________________________________________
m1_dense_2 (Dense)           (None, 100)               5100      
_________________________________________________________________
m1_dense_3 (Dense)           (None, 150)               15150     
_________________________________________________________________
m1_dense_4 (Dense)           (None, 200)               30200     
_________________________________________________________________
m1_dropout_1 (Dropout)       (None, 200)               0         
_________________________________________________________________
m1_output (Dense)            (None, 2)                 402   

# Training

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(x_test, y_test),
    callbacks=[es]
)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred_model = model.predict(x_test.astype("float"))
y_pred = to_bin(y_pred_model)
print("Test recall: {}".format(recall(y_test, y_pred)))

y_test0 = to_1D(y_test)
y_pred0 = to_1D(y_pred)
print(confusion_matrix(y_test0, y_pred0))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
model.save("models/dl/m1.h5")

# K-fold CV

In [6]:
model_params = {
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "es": es,
}

In [7]:
scores = kfold(model, model_params, x, y, 10)

Starting 10-fold cross validation:
    Validation 01 of 10 ... done [21.78s]
    Validation 02 of 10 ... done [4.04s]
    Validation 03 of 10 ... done [5.72s]
    Validation 04 of 10 ... done [6.34s]
    Validation 05 of 10 ... done [4.93s]
    Validation 06 of 10 ... done [4.56s]
    Validation 07 of 10 ... done [5.69s]
    Validation 08 of 10 ... done [4.74s]
    Validation 09 of 10 ... done [5.72s]
    Validation 10 of 10 ... done [4.61s]


In [9]:
arr = scores_dict_to_array(scores)
arr

c:\program files\python37\lib\site-packages\numba\ir_utils.py:1959: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'scores_dict' of function 'scores_dict_to_array'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "code\modelhelper.py", line 187:
@jit
def scores_dict_to_array(scores_dict):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


array([[0.95783333, 0.958     , 0.955     , 0.95916667, 0.96066667,
        0.96066667, 0.96466667, 0.96116667, 0.96216667, 0.961     ],
       [0.95315985, 0.97101449, 0.94675419, 0.95718433, 0.97505669,
        0.94142555, 0.97079772, 0.95577746, 0.95020188, 0.98026835],
       [0.87092391, 0.85608608, 0.86822742, 0.87641196, 0.86403215,
        0.89710827, 0.88852673, 0.88682991, 0.90223642, 0.85596141],
       [0.01391343, 0.00842012, 0.01620422, 0.0131257 , 0.00732194,
        0.01839131, 0.00918047, 0.01381154, 0.01668546, 0.00549571]])

In [10]:
np.save("cv/m1_cv.npy", arr)